# SETUP

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import psycopg2
from sqlalchemy import create_engine

# GET DATA

In [2]:
# fetch a web page
htlm_page = requests.get("https://www.francetvinfo.fr/politique/")
print(htlm_page.text[:1000])

<!DOCTYPE html><!--[if lt IE 7]><html class="no-js ie ie6 lte9 lte8 lte7" lang="fr"> <![endif]--><!--[if IE 7]><html class="no-js ie ie7 lte9 lte8 lte7" lang="fr"> <![endif]--><!--[if IE 8]><html class="no-js ie ie8 lte9 lte8" lang="fr"> <![endif]--><!--[if IE 9]><html class="no-js ie ie9 lte9" lang="fr"> <![endif]--><!--[if gt IE 9]><html class="no-js ie" lang="fr"> <![endif]--><!--[if !IE]><!--><html class="no-js" lang="fr"> <!--<![endif]--><head><title>Politique</title><meta http-equiv="pragma" content="no-cache" /><meta http-equiv="expires" content="Tue, 22 Feb 2000 12:14:57 GMT" /><meta http-equiv="refresh" content="1799" /><meta property="og:title" content="Politique" /><meta property="og:description" content="Suivez en direct toute l'actualité &quot;Politique&quot; : vivez l'info en live, en images et en vidéos. Participez, commentez et partager avec Franceinfo en temps réel !" /><meta property="og:type" content="website" /><meta property="og:locale:alternate" content="fr_FR" />

In [3]:
page = BeautifulSoup(htlm_page.text, "html5lib")
print(page.get_text()[:1000])

Politique  var myHash='',queryString=document.location.search;var xtref=document.referrer.replace(/[<>]/g,'').replace(/&/g,'$');myHash='xtref='+((''!=xtref)?xtref:'acc_dir');if(document.location.search&&(new RegExp("xtor=([^&]+)")).test(document.location.search)){var xtor=(new RegExp("xtor=([^&]+)")).exec(document.location.search);if(xtor){myHash=((myHash!='')?'&':'')+'xtor='+xtor[1]} console.log('document.location.search',document.location.search);var queryString=document.location.search.replace(xtor[0],'');console.log("queryString",queryString);} myHash=(document.location.hash!='')?document.location.hash+((myHash!='')?'&'+myHash:''):'#'+myHash;(function(mobileUri){var isIpad=navigator.userAgent.match(/iPad/i)!=null,isMobile="undefined"!==typeof window.screen&&"undefined"!==typeof window.screen.width&&(768>=window.screen.width);if(isMobile&&!isIpad){window.location.href="//"+window.location.hostname.replace("www","mobile")+mobileUri}})(("/politique/"+queryString+myHash));  (function()

## EXTRACT TITLES

In [4]:
# extract title
summaries = page.find_all('div', class_="flowItem")

In [5]:
summaries[0]

<div class="flowItem feature" entity="CMS-Video_3865105"><a href="/sante/maladie/coronavirus/allocution-d-emmanuel-macron-un-ton-inedit-selon-jeff-wittenberg_3865105.html"><figure id="image_3658891"><span class="videoIcon"></span>    <img alt="Emmanuel Macron, le 13 mars 2020.&amp;nbsp;" botag="CMS-IMAGE_3658891" class="b-lazy" data-src="/image/75rw8tmzo-9520/840/472/21128197.jpg" height="472px" src="data:image/gif;base64,R0lGODlhAQABAAAAACH5BAEKAAEALAAAAAABAAEAAAICTAEAOw==" title="Emmanuel Macron, le 13 mars 2020.  | LUDOVIC MARIN / AFP" width="840px"/></figure><div class="right-content"><h2 class="title" field="title">Allocution d'Emmanuel Macron : "un ton inédit", selon Jeff Wittenberg</h2><div class="description" field="description">La longue intervention d'Emmanuel Macron, dans la soirée du jeudi 12 mars, devrait rester un des moments forts de son quinquennat.</div></div></a></div>

In [6]:
summaries[0].select_one("h2").get_text()

'Allocution d\'Emmanuel Macron : "un ton inédit", selon Jeff Wittenberg'

In [7]:
len(summaries)

30

In [8]:
politic_news = [x.select_one("h2").get_text() for x in summaries]

In [9]:
politic_news

['Allocution d\'Emmanuel Macron : "un ton inédit", selon Jeff Wittenberg',
 'Coronavirus : "Le chef de l\'État envisageait vraiment de repousser les élections", affirme Nathalie Saint-Cricq',
 'Municipales : les élections auront bien lieu, malgré la menace du coronavirus',
 'RECIT. Coronavirus : le jour où Emmanuel Macron a failli reporter les élections municipales',
 'Affaire Karachi\xa0: Edouard Balladur sera jugé pour des soupçons de financement occulte de sa campagne présidentielle de 1995',
 "Désintox. L'utilisation de l’article 49.3 dans le projet de loi sur les retraites n'est pas anticonstitutionnel",
 'Chômage partiel, télétravail, déficit liés au coronavirus... Le "8h30 franceinfo" de Muriel Pénicaud',
 "Le brief éco. Coronavirus et soutien à l'économie : Emmanuel Macron renvoie la balle à Christine Lagarde",
 'Maintien des élections municipales : "Le message semble quelque peu contradictoire", estime Anne Bourse',
 'Coronavirus : les élections municipales maintenues',
 "Coro

# WRITE THE DATA IN POSTGRESQL DB

In [10]:
df = pd.DataFrame({"source": "franceinfo", "headline": politic_news, "date": pd.to_datetime('today').to_pydatetime().date()})

In [11]:
df.head()

,source,headline,date
0,franceinfo,"Allocution d'Emmanuel Macron : ""un ton inédit""...",2020-03-13
1,franceinfo,"Coronavirus : ""Le chef de l'État envisageait v...",2020-03-13
2,franceinfo,"Municipales : les élections auront bien lieu, ...",2020-03-13
3,franceinfo,RECIT. Coronavirus : le jour où Emmanuel Macro...,2020-03-13
4,franceinfo,Affaire Karachi : Edouard Balladur sera jugé p...,2020-03-13


In [16]:
# create an engine
alchemy_engine = create_engine('postgresql+psycopg2://antoinemertz@localhost/news', pool_recycle=3600)

In [17]:
conn = alchemy_engine.connect()
table = "politic"

In [19]:
already_in = pd.read_sql("SELECT * FROM  politic;", conn)
[x for x in df["headline"] if x in already_in["headline"]]

[]

In [25]:
df["headline"][0]

'Allocution d\'Emmanuel Macron : "un ton inédit", selon Jeff Wittenberg'

In [24]:
df["headline"][0] in already_in["headline"]

False

In [21]:
already_in["headline"]

0     Municipales 2020 à Paris : tout comprendre des...
1     Coronavirus : que se passera-t-il si Emmanuel ...
2     "On est tout à fait dans l'inédit" : comment l...
3     La peur du coronavirus aura-t-elle vraiment un...
4     Coronavirus, Emmanuel Macron, municipales… Le ...
5     Coronavirus : première allocution télévisée d'...
6     Coronavirus : LR et PS réclament un plan d'urg...
7     Coronavirus : "L'Union européenne n'a pas fait...
8     "On ne peut pas en faire seulement une femme t...
9     Municipales : "Fusionner avec En marche, au re...
10    Coronavirus : les députés LR déposent une prop...
11    "Un réquisitoire doit être fondé en droit et i...
12    Coronavirus : Yannick Jadot attend d'Emmanuel ...
13    Coronavirus : mobilisation générale au sommet ...
14    CARTE. Municipales 2020 : découvrez où le Rass...
15    CARTE. Municipales 2020 : découvrez où La Répu...
16    Coronavirus : visites suspendues dans les Ehpa...
17    "Ils n'ont pas le droit à l'erreur" : comm

In [14]:
try:
    frame = df.to_sql(name=table, con=conn, if_exists='append', index=False)
except Exception as ex:
    print(ex)
conn.close()

(psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "politic_pkey"
DETAIL:  Key (headline)=(Coronavirus : "Le chef de l'État envisageait vraiment de repousser les élections", affirme Nathalie Saint-Cricq) already exists.

[SQL: INSERT INTO politic (source, headline, date) VALUES (%(source)s, %(headline)s, %(date)s)]
[parameters: ({'source': 'franceinfo', 'headline': 'Allocution d\'Emmanuel Macron : "un ton inédit", selon Jeff Wittenberg', 'date': datetime.date(2020, 3, 13)}, {'source': 'franceinfo', 'headline': 'Coronavirus : "Le chef de l\'État envisageait vraiment de repousser les élections", affirme Nathalie Saint-Cricq', 'date': datetime.date(2020, 3, 13)}, {'source': 'franceinfo', 'headline': 'Municipales : les élections auront bien lieu, malgré la menace du coronavirus', 'date': datetime.date(2020, 3, 13)}, {'source': 'franceinfo', 'headline': 'RECIT. Coronavirus : le jour où Emmanuel Macron a failli reporter les élections municipales', 'date': dateti